# ***As a Tribute to Bender Bending Rodríguez***

<img src="https://upload.wikimedia.org/wikipedia/ru/a/a6/Bender_Rodriguez.png" alt="Picture of Bender Bending Rodríguez">
<p style="text-align: center;"><em>Anything Less Than Immortality Is A Complete Waste Of Time</em></p>

# Define Functions

In [ ]:
import random
import math

def load_command(bender, warehouse_id, warehouse_distance, product_type_id, products_amount):
    """ moves specified number of items from a warehouse to Bender’s cargo hold """
    commands.append("{0} L {1} {2} {3}\n".format(bender["id"], warehouse_id, product_type_id, products_amount))
    bender["available_at_step"] += bender["current_task"]["warehouse_distance"] + 1
    bender["currently_at"] = "W"
    bender["current_task"]["warehouse_distance"] = 0
    print("Bender #{0}: \33[33mI love stealing! I love taking things!\33[0m".format(bender["id"]))

# in this version unload_command is not used and not tested
def unload_command(bender, warehouse_id, warehouse_distance, product_type_id, products_amount):
    """ moves specified number of items from Bender’s cargo hold to a warehouse """
    bender["currently_at"] = "W"
    commands.append("{0} U {1} {2} {3}\n".format(bender["id"], warehouse_id, product_type_id, products_amount))
    bender["available_at_step"] += warehouse_distance + 1
    print("Bender #{0}: \33[34mI'm the good Santa! I've got toys, at very reasonable prices!\33[0m".format(bender["id"]))

def deliver_command(bender):
    """ deliver specified number of items of the specified product type to a customer """
    bender["currently_at"] == "O"
    bender["available_at_step"] += bender["current_task"]["order_distance"]
    # make sure that current order of the task is set correctly
    bender["current_task"]["order_id"] = bender["current_task"]["orders"][0]["order_id"]
    bender["current_task"]["order_distance"] = 0
    # search in Bender's loaded_products, starting from the end of the list
    for i in range(len(bender["loaded_products"]) - 1, -1, -1):
        # if the product is for current order of the task
        if bender["loaded_products"][i]["order_id"] == bender["current_task"]["order_id"]:
            # remove product from Bender's list of loaded_products
            product = bender["loaded_products"].pop()
            # remove product from Bender's cargo hold
            bender["cargo_hold"] += product_types_weights[product["type"]] * product["amount"]
            commands.append("{0} D {1} {2} {3}\n".format(bender["id"],
                                                         product["order_id"],
                                                         product["type"],
                                                         product["amount"]))
            bender["available_at_step"] += 1
            # if all products for this order are delivered
            if sum([p["amount"] for p in orders[bender["current_task"]["order_id"]]["products"]]) == 0:
                global score
                score += math.ceil(((steps_amount - bender["available_at_step"]) / steps_amount) * 100)
                print(f"\33[33m$$$ Current score: {score}\33[0m")
    if len(bender["loaded_products"]) == 0:
        # set list of warehouses, sorted by distance to Bender's current position
        bender["warehouses"] = orders[bender["current_task"]["order_id"]]["warehouses"]
        # remove Bender's current task
        bender["current_task"] = None
    # if there is no more products for this order in Bender's cargo hold 
    elif not any([bender["current_task"]["order_id"] == p["order_id"] for p in bender["loaded_products"]]):
        # remove the order from current task
        bender["current_task"]["orders"].pop(0)
        # make sure that current order of the task is set correctly
        bender["current_task"]["order_id"] = bender["current_task"]["orders"][0]["order_id"]
        bender["current_task"]["order_distance"] = bender["current_task"]["orders"][0]["order_distance"]
    print("Bender #{0}: \33[32mNow we're slaving!\33[0m".format(bender["id"]))
    
# in this version wait_command is not used and not tested
def wait_command(bender, wait_steps):
    """ command Bender to wait certain amount of steps at current location """
    commands.append("{0} W {1}\n".format(bender["id"], wait_steps))
    bender["available_at_step"] += wait_steps
        
def get_distance(row_1, col_1, row_2, col_2):
    """ get two-dimensional Euclidean distance rounded up to next integer """
    return math.ceil(math.sqrt((row_1 - row_2) ** 2 + (col_1 - col_2) ** 2))

def add_distances_to_warehouses_and_order(order):
    """ to warehouses and this order add distances between warehouses and this order """
    warehouses_for_order = []
    for warehouse in warehouses:
        distance = get_distance(order["row"], order["col"], warehouse["row"], warehouse["col"])
        warehouses_for_order.append({
            "distance": distance,
            "data": warehouse
        })
        warehouse["orders"].append({
            "distance": distance,
            "data": order
        })
    return sorted(warehouses_for_order, key = lambda w : w["distance"])

def add_distances_to_other_orders(the_order):
    """ to this order add distances to all other orders """
    orders_for_order = []
    for order in orders:
        distance = get_distance(the_order["row"], the_order["col"], order["row"], order["col"])
        orders_for_order.append({
            "distance": distance,
            "data": order
        })
    return sorted(orders_for_order, key = lambda o : o["distance"])

def add_distances_to_other_warehouses(the_warehouse):
    """ to this warehouse add distances to all other warehouses """
    warehouses_for_warehouse = []
    for warehouse in warehouses:
        distance = get_distance(the_warehouse["row"], the_warehouse["col"], warehouse["row"], warehouse["col"])
        warehouses_for_warehouse.append({
            "distance": distance,
            "data": warehouse
        })
    return sorted(warehouses_for_warehouse, key = lambda w : w["distance"])

def assign_or_update_task(bender, warehouse, warehouse_distance, order, order_distance):
    """ update Bender's task or assign task to Bender, if he don't have any """
    if bender["current_task"] == None:
        bender["current_task"] = {
            "order_id": order["id"],
            "order_distance": order_distance,
            # list of orders for delivery of products
            "orders": [{"order_id": order["id"], "order_distance": order_distance}],
            "warehouse_id": warehouse["id"],
            "warehouse_distance": warehouse_distance
        }
    # update Bender's task
    elif bender["current_task"]["order_id"] != order["id"]:
        bender["current_task"]["order_id"] = order["id"]
        bender["current_task"]["order_distance"] = order_distance
        bender["current_task"]["orders"].append({"order_id": order["id"], "order_distance": order_distance})

def load_products(bender, warehouse, warehouse_distance, order, order_distance):
    """ load products to Bender's cargo hold """
    # search in products of the order
    for product in order["products"]:
        # if there is such products at the warehouse and a place for such product at Bender's cargo hold
        # and at least one such product is still required by the order
        if (warehouse["items_amounts"][product["type"]] > 0 and
                (bender["cargo_hold"] - product_types_weights[product["type"]]) >= 0 and
                product["amount"] > 0):
            # update Bender's task or assign task to Bender, if he don't have any
            assign_or_update_task(bender, warehouse, warehouse_distance, order, order_distance)
            # amount of loaded products
            products_amount = 0
            # while there is such products at the warehouse and a place for such product at Bender's cargo hold
            # and at least one such product is still required by the order
            while (warehouse["items_amounts"][product["type"]] > 0 and
                    (bender["cargo_hold"] - product_types_weights[product["type"]]) >= 0 and
                    product["amount"] > 0):
                warehouse["items_amounts"][product["type"]] -= 1
                bender["cargo_hold"] -= product_types_weights[product["type"]]
                product["amount"] -= 1
                products_amount += 1
            # info about product loaded to Bender's cargo hold
            bender["loaded_products"].append({
                "order_id": order["id"],
                "type": product["type"],
                "amount": products_amount
            })
            load_command(bender, warehouse["id"], warehouse_distance, product["type"], products_amount)
            # if all products for this order will be delivered
            if sum([p["amount"] for p in order["products"]]) == 0:
                orders[bender["current_task"]["order_id"]]["fulfilled"] = True
            return True
    return False

def no_task_actions(bender):
    """ if possible, assign task to Bender """
    if bender["current_task"] == None:
        # search in orders starting from the closest to other orders
        for order in closest_orders:
            # if order is not yet fulfilled
            if orders[order["id"]]["fulfilled"] == False:
                for warehouse in bender["warehouses"]:
                    for warehouse_order in warehouse["data"]["orders"]:
                        if warehouse_order["data"]["id"] == order["id"]:
                            order_distance = warehouse_order["distance"]
                    # if it is possible to load some of order's products from warehouse to Bender's cargo hold
                    if load_products(
                            bender,
                            warehouse["data"],
                            warehouse["distance"],
                            orders[order["id"]],
                            order_distance
                        ):
                        return True
        # wait all remaining steps
        bender["available_at_step"] += steps_amount
        if random.random() < 0.25:
            print("Bender #{0}: \33[35mShe'll be coming 'round the mountain when she comes!\33[0m".format(bender["id"]))
        elif random.random() < 0.25:
            print("Bender #{0}: \33[35mShe'll be ridin' six white horses when she comes!\33[0m".format(bender["id"]))
        elif random.random() < 0.25:
            print("Bender #{0}: \33[36mYes I'll shoot her with my raygun when she comes!\33[0m".format(bender["id"]))
        else:
            print("Bender #{0}: \33[31mI'll be blastin' all the humans in the world!\33[0m".format(bender["id"]))
        return True
    return False

def free_space_in_cargo_hold(bender):
    """
        if there is free space in Bender's cargo hold,
        search products for orders closest to the current order
    """
    if bender["cargo_hold"] > 0:
        # search in orders closest to the current order
        for order in orders[bender["current_task"]["order_id"]]["orders"]:
            # if distance from current order (A) to the order near by (B) is lower than
            # distance from the order near by (B) to warehouse closest to the order near by (C)
            # AB < BC
            if (order["data"]["fulfilled"] == False and
                    order["distance"] < order["data"]["warehouses"][0]["distance"]):
                # if it is possible to load some of order's products from warehouse to Bender's cargo hold
                if load_products(
                        bender,
                        warehouses[bender["current_task"]["warehouse_id"]],
                        bender["current_task"]["warehouse_distance"],
                        order["data"],
                        order["distance"]
                    ):
                    return True
    return False

def at_warehouse_actions(bender):
    """ if possible, load more products for current order or the order closest to current order """
    if bender["currently_at"] == "W":
        # if it is possible to load more of current order's products
        # from warehouse to Bender's cargo hold
        if load_products(
                bender,
                warehouses[bender["current_task"]["warehouse_id"]],
                bender["current_task"]["warehouse_distance"],
                orders[bender["current_task"]["order_id"]],
                bender["current_task"]["order_distance"]
            ):
            return True
        # if there is free space in Bender's cargo hold
        elif free_space_in_cargo_hold(bender):
            return True
        # deliver current Bender's order
        deliver_command(bender)
        return True
    return False

def at_order_actions(bender):
    """ deliver loaded products """
    if bender["currently_at"] == "O":
        deliver_command(bender)
        return True
    return False
            
def write_command(bender):
    """ write command for this Bender """
    # if Bender currently not doing anything
    if no_task_actions(bender):
        return
    # if Bender is at warehouse right now
    elif at_warehouse_actions(bender):
        return
    # if Bender is at order location right now
    elif at_order_actions(bender):
        return
                        
print("Bender: Bring it on, baby!")

# Get the Data

In [ ]:
# open data file
data_file = open("/kaggle/input/hashcode-drone-delivery/busy_day.in", "r")
# get list of lines, each line is a list of integers
lines = list(map(lambda line : [int(num) for num in line.split()], data_file.read().splitlines()))
# reverse lines for better performance, to use lines.pop() instead of lines.pop(0)
lines.reverse()
# close data file
data_file.close()

rows_amount, columns_amount, benders_amount, steps_amount, bender_capacity = lines.pop()

[product_types_amount] = lines.pop()

product_types_weights = lines.pop()

[warehouses_amount] = lines.pop()

warehouses = []
for i in range(warehouses_amount):
    row, col = lines.pop()
    warehouses.append({
        "id": i,
        "row": row,
        "col": col,
        # amounts of items of each product type
        "items_amounts": lines.pop(),
        # list of orders sorted by distance from this warehouse
        "orders": []
    })
    
[orders_amount] = lines.pop()

orders = []
for i in range(orders_amount):
    row, col = lines.pop()
    orders.append({
        "id": i,
        "row": row,
        "col": col,
        # amount of items of the order
        "items_amount": lines.pop()[0],
        # types and amounts of each product of the order
        "products": [],
        # order is fulfilled
        "fulfilled": False
    })
    product_types_of_items = lines.pop()
    unique_product_types = set(product_types_of_items)
    for product_type in unique_product_types:
        orders[i]["products"].append({
            "type": product_type,
            "amount": product_types_of_items.count(product_type)
        })
    # list of warehouses sorted by distance from this order
    orders[i]["warehouses"] = add_distances_to_warehouses_and_order(orders[i])
    # list of orders sorted by distance from this order
    orders[i]["orders"] = add_distances_to_other_orders(orders[i])
    
# list of order ids
closest_orders = []
for order in orders:
    closest_orders.append({
        "id": order["id"],
        "distances_sum": sum([o["distance"] for o in order["orders"]])
    })
# sort orders by sum of distances to other orders (ascending)
closest_orders = sorted(closest_orders, key = lambda o : o["distances_sum"])
    
for warehouse in warehouses:
    # sort orders for each warehouse by distance to it
    warehouse["orders"] = sorted(warehouse["orders"], key = lambda o : o["distance"])
    # list of warehouses sorted by distance from this warehouse
    warehouse["warehouses"] = add_distances_to_other_warehouses(warehouse)
    
print("Bender: \33[33mHow can I be so bad at everything I try and still be so great?\33[0m")

# Debug Data

* Debug general data

In [ ]:
print("lines:", lines)
print("rows_amount:", rows_amount)
print("columns_amount:", columns_amount)
print("benders_amount:", benders_amount)
print("steps_amount:", steps_amount)
print("bender_capacity:", bender_capacity)

* Debug product types data

In [ ]:
print("product_types_amount:", product_types_amount)
print("product_types_weights:", product_types_weights)

* Debug warehouses data

In [ ]:
print(f"warehouses_amount: {warehouses_amount}\n")
for i in range(warehouses_amount):
    print("warehouse #{0}: row {1}, col: {2}\nitems_amounts: {3}\norders_amount: {4}\n".format(i,
                                         warehouses[i]["row"],
                                         warehouses[i]["col"],
                                         warehouses[i]["items_amounts"],
                                         len(warehouses[i]["orders"])))

* Debug orders data

In [ ]:
print(f"orders_amount: {orders_amount}\n")
for i in range(orders_amount):
    print("order #{0}: row {1}, col: {2}\nitems_amount: {3}\nproducts: {4}\n"
        "warehouses_amount: {5}\n".format(i,
                                        orders[i]["row"],
                                        orders[i]["col"],
                                        orders[i]["items_amount"],
                                        orders[i]["products"],
                                        len(orders[i]["warehouses"])))

# Create the Swarm of Benders

In [ ]:
# sums of all distances from each warehouse to each order
warehouses_sums = []
for warehouse in warehouses:
    warehouses_sums.append({
        "id": warehouse["id"],
        "distances_sum": sum([o["distance"] for o in warehouse["orders"]])
    })
# sum of all sums of distances from each warehouse to each order
warehouses_total_sum = sum([w["distances_sum"] for w in warehouses_sums])
# equal distance per warehouse
equal_part_distances = warehouses_total_sum // len(warehouses)
# equal share of Benders per warehouse
equal_part_benders = benders_amount // len(warehouses)
# list of shares of benders per warehouse
benders_shares = [{
                    "id": w["id"],
                    # amount of Benders that will start looking for products from this warehouse
                    "benders": round(equal_part_benders * (equal_part_distances / w["distances_sum"]))
                } for w in warehouses_sums]
# sort shares of Benders
benders_shares = sorted(benders_shares, key = lambda b : b["benders"])
# add remaining Benders to the most crowded warehouse
benders_shares[-1]["benders"] += benders_amount % len(warehouses)

# Benders shares index
benders_shares_i = 0

# create swarm of Benders
swarm_of_benders = []

for i in range(benders_amount):
    # loop through shares of Benders
    while benders_shares[benders_shares_i]["benders"] == 0:
        benders_shares_i += 1
    swarm_of_benders.append({
        "id": i,
        "row": warehouses[0]["row"],
        "col": warehouses[0]["col"],
        "cargo_hold": bender_capacity,
        # dicts of types and amounts of products loaded to Bender's cargo hold
        "loaded_products": [],
        # dict of data of current task, if there is any current task
        "current_task": None,
        # list of closest to this Bender warehouses
        "warehouses": warehouses[benders_shares[benders_shares_i]["id"]]["warehouses"],
        # currently located at: W -> warehouse, O -> order
        "currently_at": "W",
        # Bender will not be busy at this step
        "available_at_step": 0
    })
    benders_shares[benders_shares_i]["benders"] -= 1

print(f"Swarm of {len(swarm_of_benders)} Benders: \33[36mBite our shiny metal asses!\33[0m")

# Release the Swarm of Benders

In [ ]:
print(f"Swarm of {len(swarm_of_benders)} Benders: \33[37mWell, we're boned.\33[0m\n")

# list of commands for Benders that will be written to submission.csv
commands = []

# total score
score = 0

for current_step in range(steps_amount):
    if any([current_step >= bender["available_at_step"] for bender in swarm_of_benders]):
        print(f"Step: {current_step} ===========================================================================")
    for bender in swarm_of_benders:
        # if this Bender is not busy right now
        if current_step >= bender["available_at_step"]:
            bender["available_at_step"] = current_step
            # write command to be later used in submission file
            write_command(bender)

print(f"\nSwarm of {len(swarm_of_benders)} Benders: \33[31mMust kill all humans.\33[0m")

# Print Score and Write Submission File

In [ ]:
# print score
print(f"\33[33mFinal Score: {score}\33[0m")

# write submission file
submission = open("submission.csv", "w")

# first line in submission.csv must be total amount of commands
submission.write(str(len(commands)) + "\n")

# write commands
for command in commands:
    submission.write(command)

submission.close()

print("\nBender: Yeh, well, I'm gonna go build my own delivery company, with blackjack and hookers! In fact, forget the company!")